<a href="https://colab.research.google.com/github/WhiteFox-Lugh/Pratice-TextAnalysis/blob/master/exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk
!pip install gensim

In [3]:
import nltk
import numpy as np
import pandas as pd
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

text の引用元

全て最初の概要部分です（注釈番号は抜いています）

* text1.txt  : https://en.wikipedia.org/wiki/Canis
* text2.txt  : https://en.wikipedia.org/wiki/Wolf
* text3.txt  : https://en.wikipedia.org/wiki/Coyote
* text4.txt  : https://en.wikipedia.org/wiki/African_golden_wolf
* text5.txt  : https://en.wikipedia.org/wiki/Dhole
* text6.txt  : https://en.wikipedia.org/wiki/African_wild_dog
* text7.txt  : https://en.wikipedia.org/wiki/Red_fox
* text8.txt  : https://en.wikipedia.org/wiki/Arctic_fox
* text9.txt  : https://en.wikipedia.org/wiki/Fennec_fox
* text10.txt : https://en.wikipedia.org/wiki/Kitsune